In [1]:
import pandas as pd
import math
import re
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data_url = "./data/단향 측정 데이터_김소희.xlsx"
df_dict = pd.read_excel(data_url, sheet_name=None)
print(df_dict)

{'1-heptanol':        Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB  \
0              0.000007  1.455080e-07  6.162300e-08  6.162300e-08   
1              1.089014  1.512110e-07  6.103500e-08  6.103500e-08   
2              2.241256  1.524380e-07  6.136400e-08  6.136400e-08   
3              3.319816  1.522550e-07  6.101100e-08  6.101100e-08   
4              4.403149  1.519720e-07  6.050100e-08  6.050100e-08   
...                 ...           ...           ...           ...   
35752      42317.146907  1.355400e-07  1.547080e-06  1.547080e-06   
35753      42318.359245  1.370170e-07  1.543610e-06  1.543610e-06   
35754      42319.545124  1.374300e-07  1.543960e-06  1.543960e-06   
35755      42320.737173  1.371530e-07  1.544110e-06  1.544110e-06   
35756      42321.878927  1.372520e-07  1.534830e-06  1.534830e-06   

       Ids (A)_PMMA  Unnamed: 5  gas injection time switch  
0      3.536190e-07         NaN                 0.0    off  
1      3.527170e-07         NaN   

In [3]:
df_dict.keys()

dict_keys(['1-heptanol', '1-heptanol_2', '2-ethylfenchol', '2-ethylfenchol_2', 'decanal', 'decanal_2', 'decanal_3', 'eugenol', 'eugenol_2', 'octanal', 'octanal_2', 'cis-3-hexenol', 'cis-3-hexenol_2', '2,3,5, trimethyl pyrazine', '2,3,5, trimethyl pyrazine_2', 'geraniol', 'geraniol_2'])

## Things to beaware of the data
1. the number of rows in each sensor aren't same(Even in the same scent!)

In [4]:
scent_list = []
current = 0
for scent_set_num in range(8):
    if scent_set_num == 2:
        step = 3
    else:
        step = 2
    scent_list.append(list(df_dict.keys())[current: current + step])
    current += step
print(scent_list)

[['1-heptanol', '1-heptanol_2'], ['2-ethylfenchol', '2-ethylfenchol_2'], ['decanal', 'decanal_2', 'decanal_3'], ['eugenol', 'eugenol_2'], ['octanal', 'octanal_2'], ['cis-3-hexenol', 'cis-3-hexenol_2'], ['2,3,5, trimethyl pyrazine', '2,3,5, trimethyl pyrazine_2'], ['geraniol', 'geraniol_2']]


In [40]:
scent_dict = {}

def real_name_finder(s):
    p = re.compile("([A-Za-z0-9, -]+)([\d_])*")
    return p.search(s).group(1)

# print(scent_list)
for scent_names in scent_list:
#     print(scent_names)
    min_trial_num = math.inf
    for scent_name in scent_names:
#         print(scent_name)
        scent_data_prev = df_dict[scent_name]
# 1-heptanol has Ids (A)_PMMA column which has only 839 rows. This column has too small datas so let's get rid of it.
        if scent_name == "1-heptanol":
            scent_data_prev.drop(["Ids (A)_PMMA"], 1)
            scent_data_content = scent_data_prev.iloc[:, :4]
        else:
            scent_data_content = scent_data_prev.iloc[:, :5]
        scent_data = scent_data_content.dropna()
#         print(scent_data)
# chaning time line in to the trail num. (since we have to sum diffrent trials in to one)        
        on_off_time = scent_data_prev.iloc[:, -2:].dropna().iloc[:,0].tolist()
# scent starts to get in with first "on", so let's ignore the first "off". It is just a defalt state.
        on_off_time = on_off_time[1:]
        on_off_time += [math.inf]
        
        per_dict = {}
        for i in range(len(on_off_time)//2):
            scent_cond = (on_off_time[2*i] <= scent_data["Timestamps (sec)"]) & (scent_data["Timestamps (sec)"] < on_off_time[2*i + 2])
            per_dict[i] = scent_data[scent_cond]
        scent_dict[scent_name] = per_dict
    
    scent_min_trial = min(list(map(lambda x: max(scent_dict[x].keys()), scent_names)))
    for scent_name in scent_names:
        if real_name_finder(scent_name) != scent_name:
            print(scent_name)
            for i in range(scent_min_trial):
                if i == 2:
                    x = scent_dict[real_name_finder(scent_name)][i]
                    y = scent_dict[scent_name][i]
                original_i_data = scent_dict[real_name_finder(scent_name)][i].reset_index(drop = True)
                new_i_data = scent_dict[scent_name][i].reset_index(drop = True)
                scent_dict[real_name_finder(scent_name)][i] = pd.concat([original_i_data, new_i_data], axis = 1).dropna()
                print(f"{i}번쩨 : 원래 데이터: {original_i_data}, 추가되는 데이터: {new_i_data}, 합쳐진 데이터 {pd.concat([original_i_data, new_i_data], axis = 1).dropna()}")
                print(f"실제 dictionary의 데이터: {scent_dict[real_name_finder(scent_name)][i]}")
            del scent_dict[scent_name]

1-heptanol_2
0번쩨 : 원래 데이터:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB
0           600.681525  1.506200e-07  6.080700e-08  6.080700e-08
1           601.758567  1.500970e-07  6.098600e-08  6.098600e-08
2           602.825071  1.461190e-07  6.068800e-08  6.068800e-08
3           603.936120  1.497220e-07  5.966700e-08  5.966700e-08
4           605.037679  1.514290e-07  6.177500e-08  6.177500e-08
...                ...           ...           ...           ...
9117      10994.702554  1.328650e-07  9.474500e-08  9.474500e-08
9118      10995.837691  1.359280e-07  9.454700e-08  9.454700e-08
9119      10997.103601  1.363900e-07  9.511900e-08  9.511900e-08
9120      10998.246119  1.344190e-07  9.487300e-08  9.487300e-08
9121      10999.437104  1.362870e-07  9.491700e-08  9.491700e-08

[9122 rows x 4 columns], 추가되는 데이터:       Timestamps (sec)  Ids (A)_PVA    Ids (A)_PC    Ids (A)_PS  Ids (A)_PVCA
0           601.427993     0.000028  2.332500e-08  5.762500e-08  1.280300e-07
1

실제 dictionary의 데이터:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB  \
0         30061.073903  1.269070e-07  8.394800e-07  8.394800e-07   
1         30062.280291  1.254240e-07  8.416800e-07  8.416800e-07   
2         30063.516086  1.266530e-07  8.383500e-07  8.383500e-07   
3         30064.681426  1.267840e-07  8.408200e-07  8.408200e-07   
4         30065.894473  1.264130e-07  8.407700e-07  8.407700e-07   
...                ...           ...           ...           ...   
1677      32080.956245  1.278090e-07  1.431070e-06  1.431070e-06   
1678      32082.151192  1.284290e-07  1.430390e-06  1.430390e-06   
1679      32083.345634  1.253470e-07  1.430420e-06  1.430420e-06   
1680      32084.547829  1.281610e-07  1.428960e-06  1.428960e-06   
1681      32085.697800  1.283250e-07  1.428140e-06  1.428140e-06   

      Timestamps (sec)  Ids (A)_PVA    Ids (A)_PC    Ids (A)_PS  Ids (A)_PVCA  
0         21651.208042     0.000076  2.083600e-08  4.939300e-08  1.374600e-07  
1  

실제 dictionary의 데이터:       Timestamps (sec)   Ids (A)_SAA  Ids (A)_PAS   Ids (A)_PVB    Ids (A)_PC  \
0         10400.450278  2.488170e-07     0.000002  1.854900e-07  1.762450e-07   
1         10401.583585  2.492600e-07     0.000002  1.854880e-07  1.757160e-07   
2         10402.865559  2.458660e-07     0.000002  1.856310e-07  1.757330e-07   
3         10404.022408  2.489210e-07     0.000002  1.856350e-07  1.700010e-07   
4         10405.183936  2.490100e-07     0.000002  1.845360e-07  1.760190e-07   
...                ...           ...          ...           ...           ...   
1703      12469.495384  2.477270e-07     0.000003  1.824060e-07  1.955570e-07   
1704      12470.699710  2.462550e-07     0.000003  1.824560e-07  1.954190e-07   
1705      12471.918674  2.478430e-07     0.000003  1.824580e-07  1.950600e-07   
1706      12473.104390  2.468550e-07     0.000003  1.814900e-07  1.941220e-07   
1707      12474.312973  2.476660e-07     0.000003  1.824330e-07  1.883520e-07   

      T

0번쩨 : 원래 데이터:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB    Ids (A)_PS
0           600.037229  1.558020e-07  8.151800e-08  1.154700e-07  8.627300e-08
1           601.178109  1.560370e-07  8.146900e-08  1.154810e-07  8.599800e-08
2           602.229282  1.510800e-07  8.125400e-08  1.149980e-07  8.621500e-08
3           603.337904  1.542000e-07  8.163300e-08  1.153680e-07  8.681100e-08
4           604.481175  1.560010e-07  7.760100e-08  1.156520e-07  8.681700e-08
...                ...           ...           ...           ...           ...
3943       4995.054361  1.594200e-07  2.064230e-07  1.095330e-07  7.044400e-08
3944       4996.152066  1.595640e-07  2.002420e-07  1.094510e-07  7.064300e-08
3945       4997.345606  1.593970e-07  2.056890e-07  1.094780e-07  7.044800e-08
3946       4998.425820  1.552320e-07  2.037840e-07  1.094360e-07  7.023900e-08
3947       4999.582587  1.597820e-07  2.058380e-07  1.095590e-07  6.995100e-08

[3948 rows x 5 columns], 추가되는 데이터:   

0번쩨 : 원래 데이터:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB  \
0           600.037229  1.558020e-07  8.151800e-08  1.154700e-07   
1           601.178109  1.560370e-07  8.146900e-08  1.154810e-07   
2           602.229282  1.510800e-07  8.125400e-08  1.149980e-07   
3           603.337904  1.542000e-07  8.163300e-08  1.153680e-07   
4           604.481175  1.560010e-07  7.760100e-08  1.156520e-07   
...                ...           ...           ...           ...   
1700       2483.861606  1.657950e-07  2.555020e-07  1.119990e-07   
1701       2484.957687  1.652910e-07  2.607340e-07  1.116050e-07   
1702       2486.128291  1.608900e-07  2.603990e-07  1.119630e-07   
1703       2487.240613  1.656800e-07  2.607730e-07  1.121270e-07   
1704       2488.367232  1.654480e-07  2.554050e-07  1.117300e-07   

        Ids (A)_PS  Timestamps (sec)   Ids (A)_PVA  Ids (A)_PMMA  \
0     8.627300e-08        600.166682  5.833930e-07  1.228840e-07   
1     8.599800e-08        601.914

...

실제 dictionary의 데이터:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB  \
0          3300.533649  1.631100e-07  2.740340e-07  2.880670e-07   
1          3302.279813  1.636350e-07  2.736640e-07  2.882660e-07   
2          3303.997623  1.635510e-07  2.737600e-07  2.880130e-07   
3          3305.721385  1.635070e-07  2.740660e-07  2.880020e-07   
4          3307.443497  1.633080e-07  2.738040e-07  2.881210e-07   
...                ...           ...           ...           ...   
1704       6262.019240  1.289360e-07  2.463250e-07  2.308150e-07   
1705       6263.747742  1.290770e-07  2.463040e-07  2.307480e-07   
1706       6265.495997  1.291870e-07  2.465720e-07  2.305790e-07   
1707       6267.214639  1.290200e-07  2.462670e-07  2.304580e-07   
1708       6268.960767  1.291240e-07  2.462350e-07  2.308190e-07   

        Ids (A)_PC  Timestamps (sec)   Ids (A)_PVA  Ids (A)_PMMA  \
0     1.112060e-07       3601.377856  8.271500e-08  1.000820e-07   
1     1.111500e-07       36

0번쩨 : 원래 데이터:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB    Ids (A)_PC
0           600.982012  2.423810e-07  6.573900e-08  2.210820e-07  9.327300e-08
1           602.061179  2.424950e-07  6.573800e-08  2.208950e-07  9.319600e-08
2           603.147369  2.426500e-07  6.569100e-08  2.209910e-07  9.088300e-08
3           604.229644  2.427170e-07  6.557300e-08  2.210770e-07  9.294300e-08
4           605.315909  2.426070e-07  6.525900e-08  2.195770e-07  9.174000e-08
...                ...           ...           ...           ...           ...
2756       3594.968734  2.622840e-07  1.662290e-07  2.150240e-07  1.063190e-07
2757       3596.061079  2.614660e-07  1.676840e-07  2.138870e-07  1.043800e-07
2758       3597.145960  2.611110e-07  1.682170e-07  2.161800e-07  1.078110e-07
2759       3598.225089  2.640330e-07  1.665880e-07  2.178400e-07  1.048840e-07
2760       3599.274377  2.622920e-07  1.672340e-07  2.159870e-07  1.060310e-07

[2761 rows x 5 columns], 추가되는 데이터:   

0번쩨 : 원래 데이터:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB    Ids (A)_PC
0           601.501062  1.659160e-07  4.197200e-08  1.399470e-07  7.159600e-08
1           603.217812  1.657960e-07  4.235200e-08  1.405730e-07  7.146000e-08
2           604.933841  1.658900e-07  4.226000e-08  1.406330e-07  7.161800e-08
3           606.642034  1.660400e-07  4.279700e-08  1.410130e-07  7.144200e-08
4           608.365739  1.660060e-07  4.314000e-08  1.412650e-07  7.160500e-08
...                ...           ...           ...           ...           ...
1717       3592.030516  4.523770e-06  4.604490e-07  4.850830e-06  7.954000e-08
1718       3593.782334  4.515910e-06  4.609840e-07  4.850750e-06  7.940400e-08
1719       3595.540588  4.522670e-06  4.608770e-07  4.848040e-06  7.742700e-08
1720       3597.291864  4.512750e-06  4.591470e-07  4.847520e-06  7.983700e-08
1721       3599.097193  4.521740e-06  4.606690e-07  4.846810e-06  7.965900e-08

[1722 rows x 5 columns], 추가되는 데이터:   

1번쩨 : 원래 데이터:       Timestamps (sec)   Ids (A)_SAA  Ids (A)_PAS   Ids (A)_PVB    Ids (A)_PC
0          3600.068676  3.740710e-07     0.000001  3.430870e-07  1.356760e-07
1          3601.808888  3.759590e-07     0.000001  3.431020e-07  1.355610e-07
2          3603.565182  3.759590e-07     0.000001  3.429260e-07  1.349290e-07
3          3605.292570  3.750710e-07     0.000001  3.424740e-07  1.347640e-07
4          3607.006958  3.754440e-07     0.000002  3.435930e-07  1.378160e-07
...                ...           ...          ...           ...           ...
1726       6592.278769  2.251340e-06     0.000004  1.840710e-06  9.520300e-07
1727       6594.004147  2.251360e-06     0.000004  1.840300e-06  9.482800e-07
1728       6595.760420  2.250370e-06     0.000004  1.840050e-06  9.508300e-07
1729       6597.474859  2.250530e-06     0.000004  1.839190e-06  9.509900e-07
1730       6599.230298  2.249860e-06     0.000004  1.838370e-06  9.503200e-07

[1731 rows x 5 columns], 추가되는 데이터:       Timestam

1번쩨 : 원래 데이터:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB    Ids (A)_PC
0          3640.540078  1.377860e-07  3.479600e-08  7.655300e-08  2.899100e-08
1          3642.263285  1.373310e-07  3.489300e-08  7.657100e-08  2.916700e-08
2          3644.007539  1.378450e-07  3.500300e-08  7.655400e-08  2.850100e-08
3          3645.757881  1.378410e-07  3.500400e-08  7.658000e-08  2.910300e-08
4          3647.486326  1.376780e-07  3.467100e-08  7.612000e-08  2.768700e-08
...                ...           ...           ...           ...           ...
1703       6632.425742  7.527700e-08  5.715300e-08  3.994200e-08  2.027500e-08
1704       6634.218086  7.553500e-08  5.823800e-08  3.997700e-08  2.024700e-08
1705       6636.022373  7.521800e-08  5.815900e-08  4.001100e-08  2.008600e-08
1706       6637.765453  7.541200e-08  5.821100e-08  3.991300e-08  2.021900e-08
1707       6639.515907  7.554700e-08  5.792000e-08  3.899100e-08  2.024400e-08

[1708 rows x 5 columns], 추가되는 데이터:   

In [34]:
print(scent_dict)

{'1-heptanol': {0:       Timestamps (sec)   Ids (A)_SAA   Ids (A)_PAS   Ids (A)_PVB  \
0           600.681525  1.506200e-07  6.080700e-08  6.080700e-08   
1           601.758567  1.500970e-07  6.098600e-08  6.098600e-08   
2           602.825071  1.461190e-07  6.068800e-08  6.068800e-08   
3           603.936120  1.497220e-07  5.966700e-08  5.966700e-08   
4           605.037679  1.514290e-07  6.177500e-08  6.177500e-08   
...                ...           ...           ...           ...   
1706       2505.917621  1.493700e-07  1.060560e-07  1.060560e-07   
1707       2507.020182  1.486910e-07  1.096240e-07  1.096240e-07   
1708       2508.162533  1.493700e-07  1.099690e-07  1.099690e-07   
1709       2509.270245  1.486530e-07  1.101820e-07  1.101820e-07   
1710       2510.433497  1.490750e-07  1.060320e-07  1.060320e-07   

      Timestamps (sec)  Ids (A)_PVA    Ids (A)_PC    Ids (A)_PS  Ids (A)_PVCA  
0           601.427993     0.000028  2.332500e-08  5.762500e-08  1.280300e-07  
1   

In [39]:
for scent_name in scent_dict.keys():
    print(scent_name)
    for trial in scent_dict[scent_name].keys():
        print(scent_dict[scent_name][trial].shape)

1-heptanol
(1711, 9)
(1653, 9)
(1761, 9)
(1696, 9)
(1720, 9)
(1685, 9)
(1683, 9)
(1682, 9)
(1681, 9)
(2481, 4)
(2683, 4)
2-ethylfenchol
(1730, 10)
(1725, 10)
(1719, 10)
(1708, 10)
(1700, 10)
(1689, 10)
(1682, 10)
(2481, 5)
(2482, 5)
(2461, 5)
(4155, 5)
decanal
(1681, 15)
(1679, 15)
(1694, 15)
(670, 15)
(1674, 15)
(2527, 5)
(2533, 5)
(2560, 5)
(2561, 5)
(2543, 5)
(126, 5)
eugenol
(1713, 10)
(1709, 10)
(1705, 10)
(1707, 10)
(1698, 10)
(1703, 5)
(1697, 5)
(1691, 5)
(1686, 5)
(1657, 5)
octanal
(1716, 10)
(1702, 10)
(1709, 10)
(1704, 10)
(1708, 10)
(2611, 5)
(2609, 5)
(2604, 5)
(2590, 5)
(2611, 5)
cis-3-hexenol
(1714, 10)
(1706, 10)
(1697, 10)
(1677, 10)
(1669, 10)
(1671, 5)
(1666, 5)
(1663, 5)
(1666, 5)
(1753, 5)
2,3,5, trimethyl pyrazine
(1650, 10)
(1677, 10)
(1672, 10)
(1669, 10)
(1677, 10)
(1699, 5)
(1693, 5)
(1943, 5)
geraniol
(1704, 10)
(1696, 10)
(1689, 10)
(1679, 10)
(1682, 10)
(1681, 5)
(1674, 5)
(2843, 5)


In [38]:
print(list(scent_dict.keys()))
print(list(scent_dict["1-heptanol"].keys()))

['1-heptanol', '2-ethylfenchol', 'decanal', 'eugenol', 'octanal', 'cis-3-hexenol', '2,3,5, trimethyl pyrazine', 'geraniol']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
